In [ ]:
import requests
#from tqdm import tqdm
from tqdm.notebook import tqdm 
import sys
import os
import time

In [ ]:
# 有断点续传的版本
def downloadFILE(url,name):
    resp = requests.get(url=url,stream=True,verify=False)
    total_size = int(resp.headers['Content-Length'])

    if os.path.exists(name):
        temp_size = os.path.getsize(name)  # 本地已经下载的文件的大小
    else:
        temp_size = 0

    if temp_size < total_size:
        # 核心部分，这个是请求下载时，从本地文件已经下载过的后面下载
        headers = {'Range': 'bytes=%d-' % temp_size}  
        # 重新请求网址，加入新的请求头的
        resp = requests.get(url, stream=True, verify=False, headers=headers)

        # 下面写入文件也要注意，看到"ab"了吗？
        # "ab"表示追加形式写入文件
        with open(name, "ab") as f:
            #print("Pkg total size is:",content_size,'k,start...')
            print("文件%s的大小是：%.2fK,从%.2f开始下载..." % (name,total_size/1024,temp_size/1024))
            for data in tqdm(iterable=resp.iter_content(1024),initial=temp_size/1024,total=total_size/1024,unit='k',desc=name):
                f.write(data)


In [ ]:
# 下载闽教版英语
ce = "3b"
url = "http://www.171english.cn/ebooks/fujian/"+ce+"/mp3/"
filenames = ["unit1.mp3","unit2.mp3","unit3.mp3","unit4.mp3","unit5.mp3","unit6.mp3","unit7.mp3","unit8.mp3","review1.mp3","review2.mp3"]
#filename = "unit1.mp3"
#downloadFILE(url+filename,filename)
# filename = "review1.mp3"
# downloadFILE(url+filename,filename)
for filename in filenames:
    #filename = "unit%d.mp3"%i
    end = False
    while not end:
        try:
            downloadFILE(url+filename,ce+"/"+filename)
            end = True
            print("%s下载完成！" % filename)
        except Exception as e:
            print(filename + "下载失败！重来...", e)        
            
    

In [ ]:
def download():
    global path
    xuanzhong_index = Listbox1.curselection()
    Text1.insert(END, '> ' + str(len(xuanzhong_index)) + '个任务正在下载\n')
    Text1.see(END)
    for n in range(0,len(xuanzhong_index)):
        name = Listbox1.get(xuanzhong_index[n])
        url = Listbox2.get(xuanzhong_index[n])
        file_name = path + '\\' + name + '.mp3'
        file1 = requests.get(url,headers = headers)
        with open(file_name,'wb') as code:
            code.write(file1.content)
        Text1.insert(END, '> ' + file_name + '下载成功\n')
        Text1.see(END)

In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.96 Safari/537.36'}
#url = "http://www.171english.cn/ebooks/fujian/3a/wenben/mp3/"
url = "http://www.171english.cn/ebooks/fujian/4b/mp3/"
filename = "unit4.mp3"
file1 = requests.get(url+filename,headers=headers)
with open(filename,'wb') as code:
    code.write(file1.content)

In [ ]:
# 示例文件 来自：https://stackoverflow.com/questions/51812449/how-to-resume-file-download-in-python-3-5
def fetch_or_resume(url, filename):
    with open(filename, 'ab') as f:
        headers = {}
        pos = f.tell()
        if pos:
            headers['Range'] = f'bytes={pos}-'
        response = requests.get(url, headers=headers, stream=True)
        if pos:
            validate_as_paranoid_as_you_want_to_be_(pos, response)
        total_size = int(response.headers.get('content-length'))  
        for data in tqdm(iterable = response.iter_content(chunk_size = 1024), total = total_size//1024, unit = 'KB'):
            file.write(data)

In [ ]:
# 无断点续传的版本
def downloadFILE_old(url,name):
    try:
        resp = requests.get(url=url,headers=headers,stream=True)
        content_size = int(resp.headers['Content-Length'])/1024
        with open(name, "wb") as f:
            #print("Pkg total size is:",content_size,'k,start...')
            print("文件%s的大小是：%.2fK,开始下载..." % (name,content_size))
            for data in tqdm(iterable=resp.iter_content(1024),total=content_size,unit='k',desc=name):
                f.write(data)
        print(name + "下载完成！")
    except Exception as e:
        print(name + "下载失败！", e)

# for i in range(6,9):
#     filename = "unit%d.mp3"%i
#     downloadFILE(url+filename,filename)

In [ ]:
for i in range(1,9):
    print("\"unit%d.mp3\","%i,end="")

In [ ]:
import sys
import requests
import os

# 屏蔽warning信息
requests.packages.urllib3.disable_warnings()

def download(url, file_path):
    # 第一次请求是为了得到文件总大小
    r1 = requests.get(url, stream=True, verify=False)
    total_size = int(r1.headers['Content-Length'])

    # 这重要了，先看看本地文件下载了多少
    if os.path.exists(file_path):
        temp_size = os.path.getsize(file_path)  # 本地已经下载的文件大小
    else:
        temp_size = 0
    # 显示一下下载了多少   
    print(temp_size)
    print(total_size)
    # 核心部分，这个是请求下载时，从本地文件已经下载过的后面下载
    headers = {'Range': 'bytes=%d-' % temp_size}  
    # 重新请求网址，加入新的请求头的
    r = requests.get(url, stream=True, verify=False, headers=headers)

    # 下面写入文件也要注意，看到"ab"了吗？
    # "ab"表示追加形式写入文件
    with open(file_path, "ab") as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                temp_size += len(chunk)
                f.write(chunk)
                f.flush()

                ###这是下载实现进度显示####
                done = int(50 * temp_size / total_size)
                sys.stdout.write("\r[%s%s] %d%%" % ('█' * done, ' ' * (50 - done), 100 * temp_size / total_size))
                sys.stdout.flush()
    print()  # 避免上面\r 回车符


if __name__ == '__main__':
    link = "http://www.171english.cn/ebooks/fujian/5a/mp3/"
    filename = "unit8.mp3"
    #path = r'F:\SYY\temp\a.txt'
    #url = os.path.join(link+filename, filename)
    # 调用一下函数试试
    end = False
    while not end :
        try:
            download(link+filename, filename)
            print("download compelte!")
            end = True
        except:
            sys.stdout.flush()
            pass
